# Last-Mile Logistics Route Optimization & Analytics
**Objective:** Solving the Capacitated Vehicle Routing Problem (CVRP) for a 3-truck delivery fleet in Kolkata using K-Means spatial clustering and the Nearest Neighbor routing heuristic. 

**Impact:** Demonstrates load-balancing and dynamic distance calculation (Haversine formula) to minimize fleet fuel consumption.

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from math import radians, cos, sin, asin, sqrt

##### Calculations of real distance in kilometers between two GPS coordinates

In [2]:
def calculate_distance(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 6371 * 2 * asin(sqrt(a)) # 6371 is the Earth's radius in km

print("Loading data...")
df = pd.read_csv('deliveries.csv')
X = df[['latitude', 'longitude']]

print("Running K-Means algorithm...")
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['cluster_zone'] = kmeans.fit_predict(X)
centers = kmeans.cluster_centers_

m = folium.Map(location=[22.5726, 88.3639], zoom_start=12)
zone_colors = {0: 'red', 1: 'blue', 2: 'green'}

print("Plotting points and calculating optimal routes...\n")
print("--- FLEET ROUTING & OPERATIONS KPIs ---")

total_fleet_distance = 0

Loading data...
Running K-Means algorithm...
Plotting points and calculating optimal routes...

--- FLEET ROUTING & OPERATIONS KPIs ---


d:\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


##### THE ROUTING ENGINE

In [3]:
for i in range(3):
    # Get all delivery points for this specific truck
    zone_data = df[df['cluster_zone'] == i]
    unvisited_points = zone_data[['latitude', 'longitude']].values.tolist()
    
    # Hub location for this truck
    hub = [centers[i][0], centers[i][1]]
    
    # Plot the Hub
    folium.Marker(
        location=hub,
        icon=folium.Icon(color='black', icon='star'),
        popup=f"Zone {i} Hub"
    ).add_to(m)

    # Variables to track the truck's journey
    current_location = hub
    route_path = [hub] # Start path at the hub
    truck_distance_km = 0
    total_demand = zone_data['demand_weight'].sum()

    # Nearest Neighbor Algorithm
    while len(unvisited_points) > 0:
        # Find the closest point
        nearest_point = min(unvisited_points, key=lambda p: calculate_distance(current_location[0], current_location[1], p[0], p[1]))
        
        # Add distance and update path
        truck_distance_km += calculate_distance(current_location[0], current_location[1], nearest_point[0], nearest_point[1])
        route_path.append(nearest_point)
        
        # Plot the delivery point
        folium.CircleMarker(
            location=nearest_point,
            radius=6, color=zone_colors[i], fill=True, fill_opacity=0.7
        ).add_to(m)
        
        # Move truck to the new point and remove it from unvisited list
        current_location = nearest_point
        unvisited_points.remove(nearest_point)

    # Return to Hub
    truck_distance_km += calculate_distance(current_location[0], current_location[1], hub[0], hub[1])
    route_path.append(hub)
    
    # Draw the final route line on the map
    folium.PolyLine(route_path, color=zone_colors[i], weight=2.5, opacity=0.8).add_to(m)
    
    # Print the stats for this truck
    print(f"Truck {i} (Zone {zone_colors[i].upper()}):")
    print(f"  - Stops: {len(zone_data)}")
    print(f"  - Load:  {total_demand} kg")
    print(f"  - Route Distance: {truck_distance_km:.2f} km\n")
    
    total_fleet_distance += truck_distance_km

print(f"TOTAL FLEET DISTANCE: {total_fleet_distance:.2f} km")

m.save("optimized_routes.html")
print("\nSuccess! Open 'optimized_routes.html' in your browser.")

Truck 0 (Zone RED):
  - Stops: 15
  - Load:  87 kg
  - Route Distance: 41.70 km

Truck 1 (Zone BLUE):
  - Stops: 17
  - Load:  79 kg
  - Route Distance: 39.35 km

Truck 2 (Zone GREEN):
  - Stops: 18
  - Load:  83 kg
  - Route Distance: 53.66 km

TOTAL FLEET DISTANCE: 134.71 km

Success! Open 'optimized_routes.html' in your browser.
